In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import query_helper
# import websrcape_wiki_table

# Start by bringing items

In [2]:
df = query_helper.query_to_df('''SELECT * FROM routes''')

In [3]:
df.head()

,id,name,type,rating,stars,starVotes,pitches,location,url,longitude,latitude
0,105714722,Central Yellow Wall,"TR, Boulder",V3 R,4.4,22,0,"South Dakota,Custer State Park,Sylvan Lake,Syl...",https://www.mountainproject.com/route/10571472...,-103.564,43.8463
1,105714728,Waves,Sport,5.8,4.6,217,2,"South Dakota,The Needles Of Rushmore,Mount Rus...",https://www.mountainproject.com/route/10571472...,-103.466,43.8865
2,105714731,East Chimney Variation,Trad,5.7,4.1,40,2,"South Dakota,Custer State Park,Cathedral Spire...",https://www.mountainproject.com/route/10571473...,-103.534,43.8488
3,105714734,Conn Diagonal,Trad,5.7,4.9,151,3,"South Dakota,Custer State Park,Sylvan Lake,Out...",https://www.mountainproject.com/route/10571473...,-103.568,43.8487
4,105714737,Bolts for Bobs,Sport,5.8,3.5,125,1,"South Dakota,The Needles Of Rushmore,Mount Rus...",https://www.mountainproject.com/route/10571473...,-103.469,43.8870


In [4]:
df.shape

(36609, 11)

# Data  Cleaning

In [12]:
df['location']=df['location'].apply(lambda x: x.split(','))

In [13]:
df.head()

,id,name,type,rating,stars,starVotes,pitches,location,url,longitude,latitude
0,105714722,Central Yellow Wall,"TR, Boulder",V3 R,4.4,22,0,"[South Dakota, Custer State Park, Sylvan Lake,...",https://www.mountainproject.com/route/10571472...,-103.564,43.8463
1,105714728,Waves,Sport,5.8,4.6,217,2,"[South Dakota, The Needles Of Rushmore, Mount ...",https://www.mountainproject.com/route/10571472...,-103.466,43.8865
2,105714731,East Chimney Variation,Trad,5.7,4.1,40,2,"[South Dakota, Custer State Park, Cathedral Sp...",https://www.mountainproject.com/route/10571473...,-103.534,43.8488
3,105714734,Conn Diagonal,Trad,5.7,4.9,151,3,"[South Dakota, Custer State Park, Sylvan Lake,...",https://www.mountainproject.com/route/10571473...,-103.568,43.8487
4,105714737,Bolts for Bobs,Sport,5.8,3.5,125,1,"[South Dakota, The Needles Of Rushmore, Mount ...",https://www.mountainproject.com/route/10571473...,-103.469,43.8870


In [19]:
df.location.min()

['* In Progress', 'Rock Worship']

In [ ]:
df.location

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36609 entries, 0 to 36608
Data columns (total 11 columns):
id           36609 non-null int64
name         36609 non-null object
type         36609 non-null object
rating       36609 non-null object
stars        36609 non-null float64
starVotes    36609 non-null int64
pitches      36609 non-null int64
location     36609 non-null object
url          36609 non-null object
longitude    36609 non-null float64
latitude     36609 non-null float64
dtypes: float64(3), int64(3), object(5)
memory usage: 3.1+ MB


### Checking out Value counts

In [15]:
df.type.value_counts()

Sport                             12993
Boulder                            9809
Trad                               7358
TR                                 1383
Trad, TR                           1346
Trad, Alpine                        772
Sport, TR                           683
Trad, Sport                         562
Trad, Aid                           276
Boulder, Alpine                     211
Ice                                 159
Sport, Alpine                       131
TR, Boulder                          91
Trad, Ice                            67
Aid                                  66
Snow, Alpine                         57
Trad, Mixed, Ice                     47
Trad, Boulder                        42
Sport, Mixed                         39
Trad, Snow, Alpine                   33
Trad, Sport, TR                      32
Trad, Mixed, Ice, Alpine             31
Trad, TR, Alpine                     29
Trad, Ice, Alpine                    25
Ice, Snow, Alpine                    24


In [17]:
df[df['type']=='Boulder, Alpine']

,id,name,type,rating,stars,starVotes,pitches,location,url,longitude,latitude
4428,105837491,Breakfast Burrito,"Boulder, Alpine",V5-6,4.0,1,0,"[Colorado, Eagle/Vail/Minturn/Red Cliff, Bikin...",https://www.mountainproject.com/route/10583749...,-106.379,39.6429
4429,105837495,Sandy's Shortcut,"Boulder, Alpine",V1-,3.0,1,0,"[Colorado, Eagle/Vail/Minturn/Red Cliff, Bikin...",https://www.mountainproject.com/route/10583749...,-106.379,39.6429
7461,106051072,Highlords Traverse,"Boulder, Alpine",V1,4.0,1,0,"[Colorado, Idaho Springs, Loch Lomond, Hump Area]",https://www.mountainproject.com/route/10605107...,-105.677,39.8328
7462,106051084,Hump Traverse,"Boulder, Alpine",V1+,4.0,1,0,"[Colorado, Idaho Springs, Loch Lomond, Hump Area]",https://www.mountainproject.com/route/10605108...,-105.677,39.8328
9058,106244180,Autobot,"Boulder, Alpine",V5-,4.6,129,0,"[Colorado, Alpine Rock, RMNP - Rock, RMNP Boul...",https://www.mountainproject.com/route/10624418...,-105.661,40.3048
9059,106244207,Mikala,"Boulder, Alpine",V7-,3.4,64,0,"[Colorado, Alpine Rock, RMNP - Rock, RMNP Boul...",https://www.mountainproject.com/route/10624420...,-105.661,40.3048
9060,106244283,Revenge,"Boulder, Alpine",V6,3.4,58,0,"[Colorado, Alpine Rock, RMNP - Rock, RMNP Boul...",https://www.mountainproject.com/route/10624428...,-105.661,40.3048
9072,106245428,Bush Pilot,"Boulder, Alpine",V11,5.0,2,0,"[Colorado, Alpine Rock, RMNP - Rock, RMNP Boul...",https://www.mountainproject.com/route/10624542...,-105.660,40.3045
9394,106291605,Freaks of the Industry,"Boulder, Alpine",V13,5.0,1,0,"[Colorado, Alpine Rock, RMNP - Rock, RMNP Boul...",https://www.mountainproject.com/route/10629160...,-105.660,40.3045
10087,106390524,Tommy's Arete,"Boulder, Alpine",V7,4.9,121,0,"[Colorado, Alpine Rock, RMNP - Rock, RMNP Boul...",https://www.mountainproject.com/route/10639052...,-105.661,40.3048


In [32]:
df.stars.value_counts()

4.0    6903
3.0    4925
5.0    2773
3.5    2156
3.7    1453
3.3    1350
3.8    1306
4.5    1277
2.0    1275
3.6    1194
3.4    1082
4.3    1021
3.9     985
4.1     870
3.1     773
4.4     754
3.2     720
4.7     691
4.2     661
4.8     641
4.6     631
2.9     494
2.8     486
2.5     475
4.9     419
2.7     413
0.0     319
2.6     186
2.3     135
1.0      95
2.4      64
2.2      26
2.1      16
1.5      15
1.7       8
1.8       7
1.6       4
1.3       4
1.4       1
1.9       1
Name: stars, dtype: int64

In [33]:
df.starVotes.value_counts()

1       9191
2       4493
3       2445
4       1618
5       1182
6       1015
7        842
8        724
9        653
10       549
12       491
11       483
14       413
13       404
17       342
15       333
16       324
0        319
19       312
18       305
21       264
20       261
22       256
24       224
25       222
28       214
23       212
29       207
27       197
33       189
        ... 
1349       1
485        1
453        1
357        1
325        1
261        1
438        1
502        1
247        1
311        1
212        1
486        1
752        1
518        1
369        1
465        1
903        1
807        1
487        1
423        1
529        1
338        1
498        1
530        1
594        1
371        1
678        1
646        1
435        1
709        1
Name: starVotes, Length: 492, dtype: int64

In [25]:
minimum = 10
i =0
for areas in df['location']:
    if len(areas) < 4:
        print(areas)
        i+=1
print(i)
        


['Wyoming', 'Devils Tower', 'South and East Faces']
['Utah', 'Zion National Park', "Angel's Landing"]
['Utah', 'Zion National Park', 'Moonlight Buttress']
['Utah', 'Zion National Park', 'The Organ']
['Utah', 'Zion National Park', 'Cerberus Gendarme']
['Utah', 'Zion National Park', 'Temple of Sinawava']
['Utah', 'Zion National Park', 'Spearhead']
['Utah', 'Zion National Park', 'Leaning Wall']
['Utah', 'Zion National Park', 'Temple of Sinawava']
['Utah', 'Zion National Park', 'Temple of Sinawava']
['Utah', 'Zion National Park', 'Temple of Sinawava']
['Utah', 'Zion National Park', 'The Minotaur']
['Utah', 'Zion National Park', 'Moonlight Buttress']
['Utah', 'Zion National Park', "Angel's Landing"]
['Utah', 'Capitol Reef National Park', 'Terrible Twos Wall']
['Utah', 'Capitol Reef National Park', 'Terrible Twos Wall']
['Utah', 'Zion National Park', 'Leaning Wall']
['Utah', 'Zion National Park', 'Isaac']
['Utah', 'Zion National Park', 'Desert Shield Buttress']
['Utah', 'Zion National Park',

['Utah', 'Uinta Mountains', 'Mosquito Wall']
['Utah', 'Uinta Mountains', 'Mosquito Wall']
['Rhode Island', 'Lincoln Woods', 'Tuolumne Boulder']
['Rhode Island', 'Lincoln Woods', 'Yosemite Boulder']
['Rhode Island', 'Lincoln Woods', 'Suicide Boulder']
['Alabama', 'Horse Pens 40', 'Ten Pins Area']
['Alabama', 'Horse Pens 40', 'Ten Pins Area']
['Alabama', 'Horse Pens 40', 'Ten Pins Area']
['Alabama', 'Horse Pens 40', 'Ten Pins Area']
['Alabama', 'Horse Pens 40', 'Ten Pins Area']
['Alabama', 'Horse Pens 40', 'Ten Pins Area']
['Maryland', 'Sugarloaf Mountain', 'White Rocks']
['Rhode Island', 'Black Hut Boulder']
['Rhode Island', 'Black Hut Boulder']
['Rhode Island', 'Black Hut Boulder']
['Colorado', '10 Mile Canyon', 'White Cliff']
['Colorado', '10 Mile Canyon', 'White Cliff']
['Rhode Island', 'Lincoln Woods', 'Fred Boulder']
['Rhode Island', 'Lincoln Woods', 'Fred Boulder']
['Rhode Island', 'Lincoln Woods', 'Improbable Boulder']
['Rhode Island', 'Lincoln Woods', 'Improbable Boulder']
['Rho

['California', 'Inland Empire', 'Elephant Hill']
['California', 'Inland Empire', 'Elephant Hill']
['Minnesota', 'Banning State Park - Bouldering', "Teacher's Overlook"]
['Arizona', 'Southern Arizona', 'Stone Canyon']
['Arizona', 'Southern Arizona', 'Stone Canyon']
['Maryland', "Harper's Ferry", 'Balcony Rock']
['Maryland', "Harper's Ferry", 'Balcony Rock']
['California', 'Northeast California', 'The Mill']
['California', 'Northeast California', 'The Mill']
['New Hampshire', '*Rumney', 'The Northwest Territories']
['New Hampshire', '*Rumney', 'Main Cliff Right']
['New Hampshire', '*Rumney', 'The Northwest Territories']
['Washington', 'Southeast Cascades & Yakima', 'Horsethief Butte']
['Washington', 'Southeast Cascades & Yakima', 'Horsethief Butte']
['Massachusetts', 'Charlemont', 'The Riverblock']
['Massachusetts', 'Charlemont', 'The Bends Boulder']
['Massachusetts', 'Charlemont', 'The Riverblock']
['Massachusetts', 'Charlemont', 'The Riverblock']
['Utah', 'Capitol Reef National Park', 

In [10]:
length

5

In [5]:
word ='South Dakota,Custer State Park,Sylvan'

In [8]:
word.split(',')

['South Dakota', 'Custer State Park', 'Sylvan']